In [1]:
!pip install pdfplumber pandas sentence-transformers pinecone-client transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.8 MB/s eta 0:00:00


In [11]:
import pdfplumber
import pandas as pd

# Function to extract tables from a PDF
def extract_pnl_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        tables = []
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                # Convert table to DataFrame and reset column headers dynamically
                df = pd.DataFrame(table[1:], columns=table[0])  # Use the first row as headers
                tables.append(df)

        # Concatenate tables and reset the index
        combined_df = pd.concat(tables, ignore_index=True)
        combined_df.columns = pd.io.parsers.ParserBase({'names': combined_df.columns})._maybe_dedup_names(combined_df.columns)
        return combined_df

# Upload PDF file
from google.colab import files
uploaded = files.upload()
pdf_path = next(iter(uploaded.keys()))

# Extract P&L data
try:
    pnl_data = extract_pnl_from_pdf(pdf_path)
    print(pnl_data.head())
except Exception as e:
    print(f"Error extracting P&L data: {e}")


Saving Sample Financial Statement.pdf to Sample Financial Statement (2).pdf
Error extracting P&L data: Reindexing only valid with uniquely valued Index objects


In [16]:
!pip install --upgrade pinecone-client


In [25]:
import pinecone
from pinecone import Pinecone
from pinecone.core.client.model import IndexSpec
from sentence_transformers import SentenceTransformer

# Initialize Pinecone client
pinecone_client = Pinecone(api_key="pcsk_43xY4f_Qp8d4Q6TbUS7ay96MDsxDy8FppsSyn83Rvh4i1Ytgf3pU6eqrv2XhojoAng6aae")  # Replace with your API key

# Define the index name
index_name = "financial-qa"

# Check if the index exists; create if not
if index_name not in pinecone_client.list_indexes():
    pinecone_client.create_index(
        name=index_name,
        spec=IndexSpec(
            dimension=384,  # Embedding vector size for the model used
            metric="cosine"  # Distance metric
        )
    )

# Access the index
index = pinecone_client.Index(index_name)

# Initialize the SentenceTransformer embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to retrieve relevant data from Pinecone
def retrieve_relevant_data(query, top_k=3):
    # Convert the query to an embedding vector
    query_vector = embedding_model.encode(query, convert_to_tensor=False)
    # Query the Pinecone index
    results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    return results["matches"]

# Query the Pinecone index
query = "What is the gross profit for Q3 2024?"
retrieved_data = retrieve_relevant_data(query)

# Print the retrieved results
for match in retrieved_data:
    print(f"Score: {match['score']}, Metadata: {match['metadata']}")


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [15]:
!pip install pinecone-client==2.0.4


  Using cached pinecone_client-2.0.4-py3-none-any.whl.metadata (2.4 kB)
Using cached pinecone_client-2.0.4-py3-none-any.whl (395 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 5.0.1
    Uninstalling pinecone-client-5.0.1:
      Successfully uninstalled pinecone-client-5.0.1


In [28]:
from transformers import pipeline

# Load generative model
generator = pipeline("text2text-generation", model="t5-small")

def generate_response(query, retrieved_data):
    context = " ".join([match["metadata"]["Category"] for match in retrieved_data])
    prompt = f"Question: {query} Context: {context}"
    response = generator(prompt, max_length=50, num_return_sequences=1)
    return response[0]["generated_text"]

response = generate_response(query, retrieved_data)
print(f"Generated Response: {response}")

Device set to use cpu


NameError: name 'retrieved_data' is not defined

In [29]:
def run_pipeline(query):
    # Step 1: Retrieve relevant data
    retrieved_data = retrieve_relevant_data(query)

    # Step 2: Generate a response
    response = generate_response(query, retrieved_data)
    return response

# Example query
query = "How do the net income and operating expenses compare for Q1 2024?"
final_response = run_pipeline(query)
print(f"Final Response: {final_response}")


NameError: name 'embedding_model' is not defined

In [31]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Load pre-trained model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Prepare fine-tuning data
train_examples = [
    InputExample(texts=["What is gross profit?", "Profit after COGS"], label=1.0),
    InputExample(texts=["What is net income?", "Income after all expenses"], label=1.0),
    InputExample(texts=["What is gross profit?", "Net income"], label=0.0)
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

# Save the fine-tuned model
model.save("financial-embeddings")
print("Fine-tuning completed and model saved.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ImportError: Please install `datasets` to use this function: `pip install datasets`.

In [32]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
import torch

# Load GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Prepare fine-tuning data
qa_pairs = [
    {"question": "What is gross profit?", "answer": "Gross profit is revenue minus COGS."},
    {"question": "What is EBITDA?", "answer": "EBITDA is earnings before interest, taxes, depreciation, and amortization."}
]

# Fine-tune the model
model.train()
for epoch in range(3):  # 3 epochs
    for qa in qa_pairs:
        inputs = tokenizer(f"Q: {qa['question']} A: ", return_tensors="pt")
        labels = tokenizer(qa["answer"], return_tensors="pt")["input_ids"]
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Save fine-tuned model
model.save_pretrained("financial-gpt")
tokenizer.save_pretrained("financial-gpt")
print("Fine-tuning of GPT model completed.")


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Expected input batch_size (9) to match target batch_size (8).

In [33]:
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pinecone

# Initialize FastAPI
app = FastAPI()

# Load fine-tuned models
embedding_model = SentenceTransformer("financial-embeddings")
generator_model = GPT2LMHeadModel.from_pretrained("financial-gpt")
tokenizer = GPT2Tokenizer.from_pretrained("financial-gpt")

# Initialize Pinecone
pinecone.init(api_key="pcsk_43xY4f_Qp8d4Q6TbUS7ay96MDsxDy8FppsSyn83Rvh4i1Ytgf3pU6eqrv2XhojoAng6aae", environment="us-west1-gcp")
index_name = "financial-qa"
index = pinecone.Index(index_name)

# Input schema
class Query(BaseModel):
    question: str
    top_k: int = 3

# Routes
@app.post("/query")
def query_index(query: Query):
    # Get query embedding
    query_vector = embedding_model.encode(query.question, convert_to_tensor=False)
    # Retrieve relevant matches from Pinecone
    results = index.query(query_vector, top_k=query.top_k, include_metadata=True)

    # Use generator model to refine answers
    refined_answers = []
    for match in results["matches"]:
        context = match["metadata"].get("text", "")
        input_text = f"Q: {query.question} Context: {context} A:"
        inputs = tokenizer(input_text, return_tensors="pt")
        outputs = generator_model.generate(**inputs, max_length=100, num_return_sequences=1)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        refined_answers.append(answer)

    return {"answers": refined_answers}

# Run the app
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)


ModuleNotFoundError: No module named 'fastapi'

In [34]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
